## GRACE/GRACE-FO Harmonic Processing Program

```bash
pip3 install --user ipywidgets
jupyter nbextension enable --py --user widgetsnbextension
jupyter-notebook
```

In [ ]:
import os
import numpy as np
import matplotlib
matplotlib.rcParams['mathtext.default'] = 'regular'
matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 40
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation
import cartopy.crs as ccrs
import ipywidgets as widgets
from IPython.display import HTML

from gravity_toolkit.grace_find_months import grace_find_months
from gravity_toolkit.grace_input_months import grace_input_months
from gravity_toolkit.read_GIA_model import read_GIA_model
from gravity_toolkit.harmonics import harmonics
from gravity_toolkit.read_love_numbers import read_love_numbers
from gravity_toolkit.plm_holmes import plm_holmes
from gravity_toolkit.gauss_weights import gauss_weights
from gravity_toolkit.destripe_harmonics import destripe_harmonics
from gravity_toolkit.combine_harmonics import combine_harmonics
from gravity_toolkit.ncdf_write import ncdf_write
from gravity_toolkit.hdf5_write import hdf5_write

### Set the GRACE/GRACE-FO Data Directory

In [ ]:
# set the directory with GRACE/GRACE-FO data
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)
display(dirText)

### Set GRACE/GRACE-FO Parameters
- GRACE/GRACE-FO Processing Center
- GRACE/GRACE-FO Data Release
- GRACE/GRACE-FO Data Product
- GRACE/GRACE-FO Date Range

In [ ]:
# dropdown menu for setting processing center
# CSR: University of Texas Center for Space Research  
# GFZ: German Research Centre for Geosciences (GeoForschungsZentrum)
# JPL: Jet Propulsion Laboratory    
# CNES: French Centre National D'Etudes Spatiales
proc_list = ['CSR', 'GFZ', 'JPL', 'CNES']
proc_default = 'CSR'
procDropdown = widgets.Dropdown(
    options=proc_list,
    value=proc_default,
    description='Center:',
    disabled=False,
)

# dropdown menu for setting data release
drel_list = ['RL04', 'RL05', 'RL06']
drel_default = 'RL06'
drelDropdown = widgets.Dropdown(
    description='Release:',
    options=drel_list,
    value=drel_default,
    disabled=False,
)

# dropdown menu for setting data product
# GAA: non-tidal atmospheric correction  
# GAB: non-tidal oceanic correction  
# GAC: combined non-tidal atmospheric and oceanic correction  
# GAD: GRACE/GRACE-FO ocean bottom pressure product  
# GSM: corrected monthly GRACE/GRACE-FO static field product
dset_list = ['GAC', 'GAD', 'GSM']
dset_default = 'GSM'
dsetDropdown = widgets.Dropdown(
    description='Product:',
    options=dset_list,
    value=dset_default,    
    disabled=False,
)

# extract directory value from widget
base_dir = os.path.expanduser(dirText.value)
# find available months for data product
total_months = grace_find_months(base_dir, procDropdown.value,
    drelDropdown.value, DSET=dsetDropdown.value)
# select months to run
# https://tsutterley.github.io/data/GRACE-Months.html
options=['{0:03d}'.format(m) for m in total_months['months']]
monthsSelect = widgets.SelectMultiple(
    options=options,
    value=options,
    description='Months:',
    disabled=False
)

# function for setting the data release
def set_release(sender):
    if (procDropdown.value == 'CNES'):
        drel_list = ['RL01', 'RL02', 'RL03']
        drel_default = 'RL03'
    else:
        drel_list = ['RL04', 'RL05', 'RL06']
        drel_default = 'RL06'
    drelDropdown.options=drel_list
    drelDropdown.value=drel_default

# function for setting the data product
def set_product(sender):
    if ((procDropdown.value == 'CNES') and (drelDropdown.value == 'RL01')):
        dset_list = ['GAC', 'GSM']
    elif ((procDropdown.value == 'CNES') and (drelDropdown.value == 'RL02')):
        dset_list = ['GAA', 'GAB', 'GSM']
    elif ((procDropdown.value == 'CNES') and (drelDropdown.value == 'RL03')):
        dset_list = ['GSM']
    elif (procDropdown.value == 'CSR'):
        dset_list = ['GAC', 'GAD', 'GSM']
    else:
        dset_list = ['GAA', 'GAB', 'GAC', 'GAD', 'GSM']
    dsetDropdown.options=dset_list
    dsetDropdown.value=dset_default
    
# function for updating the available month
def update_months(sender):
    total_months = grace_find_months(base_dir, procDropdown.value,
        drelDropdown.value, DSET=dsetDropdown.value)
    options=['{0:03d}'.format(m) for m in total_months['months']]
    monthsSelect.options=options
    monthsSelect.value=options

# watch widgets for changes
procDropdown.observe(set_release)
drelDropdown.observe(set_product)
procDropdown.observe(set_product)
procDropdown.observe(update_months)
drelDropdown.observe(update_months)

# display widgets for setting GRACE/GRACE-FO parameters
widgets.VBox([procDropdown,drelDropdown,dsetDropdown,monthsSelect])

### Set Parameters for Reading GRACE/GRACE-FO Data
- Maximum Degree and Order
- Geocenter
- Oblateness
- Low Degree Zonals
- Pole Tide Correction
- Atmospheric Correction

In [ ]:
# set the spherical harmonic truncation parameters
lmax_default = {}
lmax_default['CNES'] = {'RL01':50,'RL02':50,'RL03':80}
# CSR RL04/5/6 at LMAX 60
lmax_default['CSR'] = {'RL04':60,'RL05':60,'RL06':60}
# GFZ RL04/5 at LMAX 90
# GFZ RL06 at LMAX 60
lmax_default['GFZ'] = {'RL04':90,'RL05':90,'RL06':60}
# JPL RL04/5/6 at LMAX 60
lmax_default['JPL'] = {'RL04':60,'RL05':60,'RL06':60}
# text entry for spherical harmonic degree
lmaxText = widgets.BoundedIntText(
    min=0,
    max=lmax_default[procDropdown.value][drelDropdown.value],
    value=lmax_default[procDropdown.value][drelDropdown.value],
    step=1,
    description='<i>&#8467;</i><sub>max</sub>:',
    disabled=False
)

# text entry for spherical harmonic order
mmaxText = widgets.BoundedIntText(
    min=0,
    max=lmax_default[procDropdown.value][drelDropdown.value],
    value=lmax_default[procDropdown.value][drelDropdown.value],
    step=1,    
    description='<i>m</i><sub>max</sub>:',
    disabled=False
)

# dropdown menu for setting geocenter
# Tellus: GRACE/GRACE-FO TN-13 from PO.DAAC
#    https://grace.jpl.nasa.gov/data/get-data/geocenter/
# SLR: satellite laser ranging from CSR
#    ftp://ftp.csr.utexas.edu/pub/slr/geocenter/
# SLF: Sutterley and Velicogna, Remote Sensing (2019)
#    https://www.mdpi.com/2072-4292/11/18/2108
geocenter_list = ['[none]', 'Tellus', 'SLR', 'SLF']
geocenter_default = 'SLF' if (dsetDropdown.value == 'GSM') else '[none]'
geocenterDropdown = widgets.Dropdown(
    options=geocenter_list,
    value=geocenter_default,
    description='Geocenter:',
    disabled=False,
)

# SLR C20
C20_list = ['[none]','CSR','GSFC']
C20_default = 'GSFC' if (dsetDropdown.value == 'GSM') else '[none]'
C20Dropdown = widgets.Dropdown(
    options=C20_list,
    value=C20_default,
    description='SLR C20:',
    disabled=False,
)

# SLR C30
C30_list = ['[none]','CSR','GSFC']
C30_default = 'GSFC' if (dsetDropdown.value == 'GSM') else '[none]'
C30Dropdown = widgets.Dropdown(
    options=C30_list,
    value=C30_default,
    description='SLR C30:',
    disabled=False,
)

# Pole Tide Drift (Wahr et al., 2015) for Release-5
poletide_default = True if ((drelDropdown.value == 'RL05')
    and (dsetDropdown.value == 'GSM')) else False
poletideCheckbox = widgets.Checkbox(
    value=poletide_default,
    description='Pole Tide Corrections',
    disabled=False
)


# ECMWF Atmospheric Jump Corrections for Release-5
atm_default = True if (dsetDropdown.value == 'RL05') else False
atmCheckbox = widgets.Checkbox(
    value=atm_default,
    description='ATM Corrections',
    disabled=False
)

# functions for setting the spherical harmonic truncation
def set_SHdegree(sender):
    lmaxText.max=lmax_default[procDropdown.value][drelDropdown.value]
    lmaxText.value=lmax_default[procDropdown.value][drelDropdown.value]

def set_SHorder(sender):
    mmaxText.max=lmaxText.value
    mmaxText.value=lmaxText.value

# functions for setting pole tide and atmospheric corrections for Release-5
def set_pole_tide(sender):
    poletideCheckbox.value = True if ((drelDropdown.value == 'RL05')
        and (dsetDropdown.value == 'GSM')) else False

def set_atm_corr(sender):
    atmCheckbox.value = True if (drelDropdown.value == 'RL05') else False

# watch processing center widget for changes
procDropdown.observe(set_SHdegree)
# watch data release widget for changes
drelDropdown.observe(set_SHdegree)
drelDropdown.observe(set_pole_tide)
drelDropdown.observe(set_atm_corr)
# watch data product widget for changes
dsetDropdown.observe(set_pole_tide)
# watch spherical harmonic degree widget for changes
lmaxText.observe(set_SHorder)
        
# display widgets for setting GRACE/GRACE-FO read parameters
widgets.VBox([lmaxText,mmaxText,geocenterDropdown,
    C20Dropdown,C30Dropdown,poletideCheckbox,atmCheckbox])

### Read GRACE/GRACE-FO data
- Extract Data Parameters

In [ ]:
# extract values from widgets
PROC = procDropdown.value
DREL = drelDropdown.value
DSET = dsetDropdown.value
months = [int(m) for m in monthsSelect.value]
LMAX = lmaxText.value
MMAX = mmaxText.value
DEG1 = geocenterDropdown.value
SLR_C20 = C20Dropdown.value
SLR_C30 = C30Dropdown.value
POLE_TIDE = poletideCheckbox.value
ATM = atmCheckbox.value

# read GRACE/GRACE-FO data for parameters
start_mon = np.min(months)
end_mon = np.max(months)
missing = sorted(set(np.arange(start_mon,end_mon+1)) - set(months))
GRACE_Ylms = grace_input_months(base_dir, PROC, DREL, DSET,
    LMAX, start_mon, end_mon, missing, SLR_C20, DEG1,
    MMAX=MMAX, SLR_C30=SLR_C30, POLE_TIDE=POLE_TIDE,
    ATM=ATM, MEAN=True)
# mid-date times in year-decimal
tdec = GRACE_Ylms['time'].copy()
# number of time steps
nt = len(months)
# flag for spherical harmonic order
order_str = 'M{0:d}'.format(MMAX) if (MMAX != LMAX) else ''

### Set Parameters to Run Specific Analyses
- GIA Correction
- Remove Specific Harmonic Fields
- Gaussian Smoothing Radius in kilometers  
- Filter (destripe) harmonics (Swenson et al., 2006)  
- Spatial degree spacing  
- Spatial degree interval  
1) (-180:180,90:-90)  
2) (degree spacing)/2  
- Output spatial units  
1) equivalent water thickness (cm)  
2) geoid height (mm)  
3) elastic crustal deformation (mm)  
4) gravitational perturbation (&mu;Gal)   
5) equivalent surface pressure (Pa)    

In [ ]:
# set the GIA file
# files come in different formats depending on the group
giaText = widgets.Text(
    value='',
    description='GIA File:',
    disabled=False
)

# dropdown menu for setting GIA model
# IJ05-R2: Ivins R2 GIA Models
# W12a: Whitehouse GIA Models
# SM09: Simpson/Milne GIA Models
# ICE6G: ICE-6G GIA Models
# Wu10: Wu (2010) GIA Correction
# AW13-ICE6G: Geruo A ICE-6G GIA Models
# Caron: Caron JPL GIA Assimilation
# ICE6G-D: ICE-6G Version-D GIA Models
# ascii: GIA reformatted to ascii
# netCDF4: GIA reformatted to netCDF4
# HDF5: GIA reformatted to HDF5
gia_list = ['[None]','IJ05-R2','W12a','SM09','ICE6G',
    'Wu10','AW13-ICE6G','Caron','ICE6G-D',
    'ascii','netCDF4','HDF5']
gia_default = '[None]'
giaDropdown = widgets.Dropdown(
    options=gia_list,
    value=gia_default,
    description='GIA Type:',
    disabled=False,
)

# set the files to be removed
removeText = widgets.Text(
    value='',
    description='Rem. Files:',
    disabled=False
)

# dropdown menu for setting remove file type
# netCDF4: single netCDF4 file
# HDF5: single HDF5 file
# index (ascii): index of monthly ascii files
# index (netCDF4): index of monthly netCDF4 files
# index (HDF5): index of monthly HDF5 files
remove_list = ['[None]','netCDF4','HDF5',
    'index (ascii)','index (netCDF4)','index (HDF5)']
remove_default = '[None]'
removeDropdown = widgets.Dropdown(
    options=remove_list,
    value=remove_default,
    description='Rem. Type:',
    disabled=False,
)

# text entry for Gaussian Smoothing Radius in km
gaussianText = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1000.0,
    step=50,
    description='Gaussian:',
    disabled=False
)

# Destripe Spherical Harmonics
destripeCheckbox = widgets.Checkbox(
    value=False,
    description='Destripe',
    disabled=False
)

# text entry for Degree Spacing
spacingText = widgets.BoundedFloatText(
    value=1.0,
    min=0,
    max=360.0,
    step=0.5,
    description='Spacing:',
    disabled=False
)

# dropdown menu for setting output data format
interval_list = ['(-180:180,90:-90)', '(Degree spacing)/2']
interval_default = '(Degree spacing)/2'
intervalDropdown = widgets.Dropdown(
    options=interval_list,
    value=interval_default,
    description='Interval:',
    disabled=False,
)

# dropdown menu for setting units
# 1: cm of water thickness
# 2: mm of geoid height
# 3: mm of elastic crustal deformation
# 4: microGal gravitational perturbation
# 5: millibar of equivalent surface pressure
unit_list = ['cmwe', 'mmGH', 'mmCU', u'\u03BCGal', 'mbar']
unit_label = ['cm', 'mm', 'mm', u'\u03BCGal', 'mb']
unit_name = ['Equivalent Water Thickness', 'Geoid Height',
    'Elastic Crustal Uplift', 'Gravitational Undulation',
    'Equivalent Surface Pressure']
unit_default = 'cmwe'
unitsDropdown = widgets.Dropdown(
    options=unit_list,
    value=unit_default,
    description='Units:',
    disabled=False,
)

# dropdown menu for setting output data format
format_list = ['[None]','netCDF4', 'HDF5']
format_default = '[None]'
formatDropdown = widgets.Dropdown(
    options=format_list,
    value=format_default,
    description='Format:',
    disabled=False,
)

# display widgets for setting GRACE/GRACE-FO read parameters
widgets.VBox([giaText,giaDropdown,removeText,removeDropdown,
    gaussianText,destripeCheckbox,spacingText,intervalDropdown,
    unitsDropdown,formatDropdown])

In [ ]:
# Output degree spacing
dlon = spacingText.value
dlat = spacingText.value
# Output Degree Interval
INTERVAL = intervalDropdown.index + 1
if (INTERVAL == 1):
    # (-180:180,90:-90)
    nlon = np.int((360.0/dlon)+1.0)
    nlat = np.int((180.0/dlat)+1.0)
    glon = -180 + dlon*np.arange(0,nlon)
    glat = 90.0 - dlat*np.arange(0,nlat)
elif (INTERVAL == 2):
    # (Degree spacing)/2
    glon = np.arange(-180+dlon/2.0,180+dlon/2.0,dlon)
    glat = np.arange(90.0-dlat/2.0,-90.0-dlat/2.0,-dlat)
    nlon = len(glon)
    nlat = len(glat)

# Computing plms for converting to spatial domain
theta = (90.0-glat)*np.pi/180.0
PLM,dPLM = plm_holmes(LMAX,np.cos(theta))

# read GIA data
GIA = giaDropdown.value
GIA_Ylms = read_GIA_model(giaText.value, GIA=GIA, LMAX=LMAX)
gia_str = '' if (GIA == '[None]') else '_{0}'.format(GIA_Ylms['title'])

# read data to be removed from GRACE/GRACE-FO monthly harmonics
remove_Ylms = {}
remove_Ylms['clm'] = np.zeros((LMAX+1,MMAX+1,nt))
remove_Ylms['slm'] = np.zeros((LMAX+1,MMAX+1,nt))
for f in removeText.value.split(','):
    if (removeDropdown.value == 'netCDF4'):
        # read netCDF4 file
        Ylms = harmonics().from_netCDF4(f)
    elif (removeDropdown.value == 'HDF5'):
        # read HDF5 file
        Ylms = harmonics().from_HDF5(f)
    elif (removeDropdown.value == 'index (ascii)'):
        # read index of ascii files
        Ylms = harmonics().from_index(f,'ascii')
    elif (removeDropdown.value == 'index (netCDF4)'):
        # read index of netCDF4 files
        Ylms = harmonics().from_index(f,'netCDF4')
    elif (removeDropdown.value == 'index (HDF5)'):
        # read index of HDF5 files
        Ylms = harmonics().from_index(f,'HDF5')
    # reduce to months of interest and truncate to range
    Ylms = Ylms.subset(months).truncate(LMAX,mmax=MMAX)
    remove_Ylms['clm'] += Ylms.clm
    remove_Ylms['slm'] += Ylms.slm

# read load love numbers file
love_numbers_file = os.path.join(base_dir,'love_numbers')
# LMAX of load love numbers from Han and Wahr (1995) is 696.
# from Wahr (2007) linearly interpolating kl works
# however, as we are linearly extrapolating out, do not make
# LMAX too much larger than 696
if (LMAX > 696):
    # Creates arrays of kl, hl, and ll Love Numbers
    hl = np.zeros((LMAX+1))
    kl = np.zeros((LMAX+1))
    ll = np.zeros((LMAX+1))
    hl[:697],kl[:697],ll[:697] = read_love_numbers(love_numbers_file,
        REFERENCE='CF', FORMAT='tuple')
    # for degrees greater than 696
    for l in range(697,LMAX+1):
        hl[l] = 2.0*hl[l-1] - hl[l-2]# linearly extrapolating hl
        kl[l] = 2.0*kl[l-1] - kl[l-2]# linearly extrapolating kl
        ll[l] = 2.0*ll[l-1] - ll[l-2]# linearly extrapolating ll
else:
    # read arrays of kl, hl, and ll Love Numbers
    hl,kl,ll = read_love_numbers(love_numbers_file, REFERENCE='CF')

# gaussian smoothing radius in km (Jekeli, 1981)
RAD = gaussianText.value
if (RAD != 0):
    wt = 2.0*np.pi*gauss_weights(RAD,LMAX)
    gw_str = '_r{0:0.0f}km'.format(RAD)
else:
    # else = 1
    wt = np.ones((LMAX+1))
    gw_str = ''

# destriping the GRACE/GRACE-FO harmonics
ds_str = '_FL' if destripeCheckbox.value else ''
    
# Earth Parameters
# Average Density of the Earth [g/cm^3]
rho_e = 5.517
# Average Radius of the Earth [cm]
rad_e = 6.371e8
# WGS84 Gravitational Constant of the Earth [cm^3/s^2]
GM_e = 3986004.418e14
# Gravitational Constant of the Earth's atmosphere
GM_atm = 3.5e14
# Gravitational Constant of the Earth (w/o atm)
GM = GM_e - GM_atm
# standard gravitational acceleration (World Meteorological Organization)
g_wmo = 9.80665

# Setting units factor for output
UNITS = unitsDropdown.index + 1
# dfactor computes the degree dependent coefficients
l = np.arange(0,LMAX+1)
if (UNITS == 1):
    # 1: cmwe, centimeters water equivalent
    dfactor = rho_e*rad_e*(2.0*l+1.0)/(1.0 +kl[l])/3.0
elif (UNITS == 2):
    # 2: mmGH, millimeters geoid height
    dfactor = np.ones(LMAX+1)*(10.0*rad_e)
elif (UNITS == 3):
    # 3: mmCU, millimeters elastic crustal deformation
    dfactor = 10.0*rad_e*hl[l]/(1.0 +kl[l])
elif (UNITS == 4):
    # 4: micGal, microGal gravity perturbations
    dfactor = 1.e6*GM*(l+1.0)/(rad_e**2.0)
elif (UNITS == 5):
    # 5: mbar, millibar equivalent surface pressure
    dfactor = g_wmo*rho_e*rad_e*(2.0*l+1.0)/(1.0 +kl[l])/30.0

# output spatial grid
spatial = np.zeros((nlat,nlon,nt))
# converting harmonics to truncated, smoothed coefficients in units
# combining harmonics to calculate output spatial fields
for i,t in enumerate(tdec):
    # spherical harmonics for time t
    clm = np.zeros((LMAX+1,MMAX+1))
    slm = np.zeros((LMAX+1,MMAX+1))
    # GRACE/GRACE-FO harmonics for time t
    # and monthly files to be removed
    if destripeCheckbox.value:
        Ylms = destripe_harmonics(GRACE_Ylms['clm'][:,:,i], 
            GRACE_Ylms['slm'][:,:,i], LMAX=LMAX, MMAX=MMAX)
        grace_clm = Ylms['clm'][:,:MMAX+1].copy()
        grace_slm = Ylms['slm'][:,:MMAX+1].copy()
        Ylms = destripe_harmonics(remove_Ylms['clm'][:,:,i], 
            remove_Ylms['slm'][:,:,i], LMAX=LMAX, MMAX=MMAX)
        remove_clm = Ylms['clm'][:,:MMAX+1].copy()
        remove_slm = Ylms['slm'][:,:MMAX+1].copy()
    else:
        grace_clm = GRACE_Ylms['clm'][:,:MMAX+1,i].copy()
        grace_slm = GRACE_Ylms['slm'][:,:MMAX+1,i].copy()
        remove_clm = remove_Ylms['clm'][:,:MMAX+1,i].copy()
        remove_slm = remove_Ylms['slm'][:,:MMAX+1,i].copy()
    # monthly GIA calculated by gia_rate*time elapsed
    gia_clm = GIA_Ylms['clm'][:,:MMAX+1]*(t-2003.3)
    gia_slm = GIA_Ylms['slm'][:,:MMAX+1]*(t-2003.3)
    # convert harmonics to output units
    # for each spherical harmonic degree
    for l in range(LMAX+1):# LMAX+1 to include LMAX
        clm[l,:] = (grace_clm[l,:] - gia_clm[l,:] -
            remove_clm[l,:])*dfactor[l]*wt[l]
        slm[l,:] = (grace_slm[l,:] - gia_slm[l,:] -
            remove_slm[l,:])*dfactor[l]*wt[l]
    # convert spherical harmonics to output spatial grid
    spatial[:,:,i] = combine_harmonics(clm, slm, glon, glat,
        LMAX=LMAX, MMAX=MMAX, PLM=PLM).T

# output to netCDF4 or HDF5
file_format = '{0}_{1}_{2}{3}{4}_{5}_L{6:d}{7}{8}{9}_{10:03d}-{11:03d}.{12}'
if (formatDropdown.value == 'netCDF4'):
    args = (PROC,DREL,DSET,gia_str,GRACE_Ylms['title'],unit_list[UNITS-1],
        LMAX,order_str,gw_str,ds_str,months[0],months[-1],'nc')
    FILE = os.path.join(GRACE_Ylms['directory'],file_format.format(*args))
    ncdf_write(spatial, glon, glat, tdec, FILENAME=FILE,
        VARNAME='z', LONNAME='lon', LATNAME='lat', TIMENAME='time',
        UNITS=unit_list[UNITS-1], LONGNAME=unit_name[UNITS-1],
        TIME_UNITS='year', TIME_LONGNAME='Date_in_Decimal_Years',
        TITLE='GRACE/GRACE-FO Spatial Data', VERBOSE=True)
elif (formatDropdown.value == 'HDF5'):
    args = (PROC,DREL,DSET,gia_str,GRACE_Ylms['title'],unit_list[UNITS-1],
        LMAX,order_str,gw_str,ds_str,months[0],months[-1],'H5')
    FILE = os.path.join(GRACE_Ylms['directory'],file_format.format(*args))    
    hdf5_write(spatial, glon, glat, tdec, FILENAME=FILE,
        VARNAME='z', LONNAME='lon', LATNAME='lat', TIMENAME='time',
        UNITS=unit_list[UNITS-1], LONGNAME=unit_name[UNITS-1],
        TIME_UNITS='year', TIME_LONGNAME='Date_in_Decimal_Years',
        TITLE='GRACE/GRACE-FO Spatial Data', VERBOSE=True)

### Set parameters for creating animation

In [ ]:
# slider for the plot min and max for normalization
vmin = np.min(spatial).astype(np.int)
vmax = np.ceil(np.max(spatial)).astype(np.int)
rangeSlider = widgets.IntRangeSlider(
    value=[vmin,vmax],
    min=vmin,
    max=vmax,
    step=1,
    description='Plot Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

# slider for steps in color bar
stepSlider = widgets.IntSlider(
    value=1,
    min=0,
    max=vmax-vmin,
    step=1,
    description='Plot Step:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

# all listed color maps in matplotlib version
cmap_set = set(plt.cm.datad.keys()) | set(plt.cm.cmaps_listed.keys())
# color maps available in this program
# (no reversed, qualitative or miscellaneous)
cmaps = {}
cmaps['Perceptually Uniform Sequential'] = ['viridis',
    'plasma','inferno','magma','cividis']
cmaps['Sequential'] = ['Greys','Purples','Blues','Greens',
    'Oranges','Reds','YlOrBr','YlOrRd','OrRd','PuRd','RdPu',
    'BuPu','GnBu','PuBu','YlGnBu','PuBuGn','BuGn','YlGn']
cmaps['Sequential (2)'] = ['binary','gist_yarg','gist_gray', 
    'gray','bone','pink','spring','summer','autumn','winter',
    'cool','Wistia','hot','afmhot','gist_heat','copper']
cmaps['Diverging'] = ['PiYG','PRGn','BrBG','PuOr','RdGy','RdBu',
    'RdYlBu','RdYlGn','Spectral','coolwarm', 'bwr','seismic']
cmaps['Cyclic'] = ['twilight','twilight_shifted','hsv']
# create list of available color maps in program
cmap_list = []
for key,val in cmaps.items():
    cmap_list.extend(val)
# reduce color maps to available in program and matplotlib
cmap_set &= set(cmap_list)
# dropdown menu for setting color map
cmapDropdown = widgets.Dropdown(
    options=sorted(cmap_set),
    value='viridis',
    description='Color Map:',
    disabled=False,
)

# Reverse the color map
cmapCheckbox = widgets.Checkbox(
    value=False,
    description='Reverse Color Map',
    disabled=False
)

# display widgets for setting GRACE/GRACE-FO plot parameters
widgets.VBox([rangeSlider,stepSlider,cmapDropdown,cmapCheckbox])

### Create animation of GRACE/GRACE-FO months

In [ ]:
fig, ax1 = plt.subplots(num=1, nrows=1, ncols=1, figsize=(10.375,6.625),
    subplot_kw=dict(projection=ccrs.PlateCarree()))

# levels and normalization for plot range
vmin,vmax = rangeSlider.value
levels = np.arange(vmin,vmax+stepSlider.value,stepSlider.value)
norm = colors.Normalize(vmin=vmin,vmax=vmax)
cmap_reverse_flag = '_r' if cmapCheckbox.value else ''
cmap = plt.cm.get_cmap(cmapDropdown.value + cmap_reverse_flag)
im = ax1.imshow(np.zeros((nlat,nlon)), interpolation='nearest',
    norm=norm, cmap=cmap, transform=ccrs.PlateCarree(),
    extent=(-180,180,-90,90), origin='upper', animated=True)
ax1.coastlines('50m')

# add date label (year-calendar month e.g. 2002-01)
time_text = ax1.text(0.025, 0.025, '', transform=fig.transFigure,
    color='k', size=24, ha='left', va='baseline')

# Add horizontal colorbar and adjust size
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# pad = distance from main plot axis
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax1, extend='both', extendfrac=0.0375,
    orientation='horizontal', pad=0.025, shrink=0.85,
    aspect=22, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_xlabel(unit_name[UNITS-1], labelpad=10, fontsize=24)
cbar.ax.set_ylabel(unit_label[UNITS-1], fontsize=24, rotation=0)
cbar.ax.yaxis.set_label_coords(1.045, 0.1)
# Set the tick levels for the colorbar
cbar.set_ticks(levels)
cbar.set_ticklabels(['{0:d}'.format(ct) for ct in levels])
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=26, labelsize=24,
    direction='in')
    
# stronger linewidth on frame
ax1.outline_patch.set_linewidth(2.0)
ax1.outline_patch.set_capstyle('projecting')
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
    
# animate frames
def animate_frames(i):
    # set image
    im.set_data(spatial[:,:,i])
    # add date label (year-calendar month e.g. 2002-01)
    year = np.floor(tdec[i]).astype(np.int)
    month = np.int((months[i]-1) % 12) + 1
    time_text.set_text(u'{0:4d}\u2013{1:02d}'.format(year,month))

# set animation
anim = animation.FuncAnimation(fig, animate_frames, frames=nt)
%matplotlib inline
HTML(anim.to_jshtml())